### Required Imports

In [ ]:
import sys, nibabel as nib, numpy as np
sys.path.insert(0, 'core/')
from epi import  data_prep_ml
from utils import imask_ut, outlier
from denoiser import cnn
from sklearn.model_selection import train_test_split
from skimage.draw import circle
from skimage.transform import hough_circle, hough_circle_peaks
from keras.callbacks import ModelCheckpoint
from scipy.stats import pearsonr
import pymc3 as pm

### Set Path for Input Files

In [ ]:
ground_truth_ts = " " # Path of the ground truth in nifti format.
measured_fmri_ts = " " # Path of the extracted measured fMRI time series in nifti format. 
masks = " " # Path of the saved masks of the extracted slices. 

### Obtain the Ground Truth, Measured fMRI and Noise

In [ ]:
measured_fmri = nib.load(measured_fmri_ts)
ground_truth = nib.load(ground_truth_ts)
imask = nib.load(masks)
imask_utils = imask_ut(imask)

stack_scn, stack_sim, noise, stack_scn_flip, stack_sim_flip, noise_flip = data_prep_ml(ground_truth,measured_fmri,imask_utils,1,600)

## stack_scn refers to the stack of measured fMRI time series
## stack_sim refers to the stack of ground truth time series

### Remove Outliers at 3 SD's away

In [ ]:
index = outlier(stack_sim,3)
scn =  np.delete(stack_scn,index,axis=0)  ## Measured fMRI after removing outlier voxels
sim =  np.delete(stack_sim,index,axis=0)  ## Ground-Truth fMRI after removing outlier voxels

# Calculate Data Quality Metrics

### Calculate Signal-to-Noise Ratio

In [ ]:
parseval_fx = np.sum(np.power(sim.flatten(),2))
parseval_fn = np.sum(np.power(scn.flatten()-sim.flatten(),2))
print('SNR: ',parseval_fx/parseval_fn)

### Calculate Dynamic Fidelity 

In [ ]:
print('Fidelity: ',pearsonr(scn.flatten(),sim.flatten()))

### Calculate Scanner Instability

In [ ]:
gt_data = sim.flatten()
fmri_data = scn.flatten()

# here we assume that the two distributions add
with pm.Model() as multi_noise:
    sigma1 = pm.Uniform('sigma1',0,100)
    ampl_noise = pm.Uniform('ampl_noise',0,100)
    
    fmri_observed = pm.Normal('fmri_observed',
                              mu=gt_data,
                              sd=np.sqrt(sigma1**2+ ampl_noise**2*gt_data**2), observed=fmri_data)

    posterior = pm.sample(njobs=4)

In [ ]:
pm.traceplot(posterior)

In [ ]:
pm.summary(posterior)

In [ ]:
thermal = np.mean(posterior['sigma1'])
beta = np.mean(posterior['ampl_noise'])
sigma_mult = np.sqrt(np.sum((beta**2)*gt_data**2)/len(gt_data))
sig_total = np.sqrt(thermal**2+sigma_mult**2)
print("sigma thermal normalized",thermal/np.std(gt_data))
print("sigma thermal normalized error",np.std(posterior['sigma1'])/np.std(gt_data))
print("sigma multi/thermal:",sigma_mult**2/sig_total**2)     # Scanner-Instability to Thermal Noise Ratio